# 仓库列表

## 1）issue

[仓库-简道云触发器 ](https://dev.eastwestec.com/projects/cecp/wiki/%E4%BB%93%E5%BA%93%E5%88%97%E8%A1%A8-%E7%AE%80%E9%81%93%E4%BA%91%E8%A7%A6%E5%8F%91%E5%99%A8)
>cecp 创建仓库：设置 -> 仓库设置，即：`setting/stock/list/add`

## 2）分支

weld_backend: `feature/0426_purchase_requisition`  
switch: `feature/0426_purchase_requisition`  

## 3）服务

### 3.1）weld_backend 与 switch 服务启动

### 3.2）celery

weld_backend: `celery -A celery_app.celery worker`  
switch: `celery -A celery_app.celery worker`  

## 4）代码解析

### 4.1）switch

portal 3件套

###  4.2）weld_backend

* a. 链接中间件接口  
`bapi.services.middlewares.api.jiandaoyun.warehouse.models.Warehouse`
  
* b. 注册：  
`bapi.services.middlewares.client.JiandaoyunMixClient`
  
* c. celery任务：  
`bapi.modules.stock.location.tasks.webhook_sync_warehouse`
  
* d. 创建后执行同步任务  
`bapi.modules.stock.location.views.StockLocationWarehouseAPIView#after_obj_created`
  
* e. 更新后执行同步任务  
`bapi.modules.stock.location.views.StockLocationWarehouseAPIView#before_update_saved`



## 5）测试

cecp / switch 可开启 debug 模式。

### 5.1）从 weld_backend 发起同步请求
  
`python manager.py shell` 

```python
# 创建
from flask_babel import lazy_gettext
from flask import request, json
from bapi.modules.stock.location.serializers import StockLocationWarehouseSerializer
from bapi.modules.stock.location.schemas import WarehouseTimeZoneSimpleSchema
from bapi.modules.stock.location.tasks import webhook_sync_warehouse
al = StockLocationWarehouse.query.all()
obj = al[0]
data = StockLocationWarehouseSerializer().dump(obj).data
data["type_name"] = unicode(lazy_gettext(data.get("type_name"))) if data.get("type_name") else None
webhook_sync_warehouse(data=json.dumps(data), action='create')



# 更新
from flask import request, json
from bapi.modules.stock.location.schemas import StockLocationWarehouseUpdateSchema
from bapi.modules.stock.location.tasks import webhook_sync_warehouse
al = StockLocationWarehouse.query.all()
obj = al[0]
data = StockLocationWarehouseUpdateSchema().dump(obj).data
data["identifier"] = obj.identifier
webhook_sync_warehouse(data=json.dumps(data), action='update')
```

### 6）注意点：

#### 翻译支持：
命令行运行：```pybabel compile -d bapi/translations```

#### 获得翻译后的数据：

```python
from flask_babel import lazy_gettext
unicode(lazy_gettext(data.get("type_name")))
```
---
---
---


# 仓库列表（优化）

## 1）issue

[仓库-简道云触发器 ](https://dev.eastwestec.com/issues/3060)
>cecp 创建亚马逊账号间接创建FBA仓库：设置 -> 账号管理 -> 添加 Amazon

## 2）分支

weld_backend: `feature/3060_warehouse_fba_sync`  
switch: `master`  

## 3）代码分析

`bapi.modules.account.views.AmazonAccountAPIView#after_obj_created`
```python
# 触发创建 简道云 仓库表
db.session.flush()  # 不然数据未插入数据库，下一步序列化会出故障
data = StockLocationWarehouseSerializer().dump(obj.warehouse).data
data["type_name"] = unicode(lazy_gettext(data.get("type_name"))) if data.get("type_name") else None
webhook_sync_warehouse.delay(data=json.dumps(data), action=kwargs.get('action', 'create'))
```
       
## 4）注意点

#### a）Amazon账号的禁用跟FBA仓库的禁用没关系。

#### b）channel_id 为 2 的是亚马逊账号
`a = Account.query.filter(Account.channel_id == 2).first()`


#### c）在这里创建 FBA仓库
`bapi.modules.account.views.AmazonAccountAPIView#after_obj_created`

#### d）mws token 

`amzn.mws.\w{8}-\w{4}-\w{4}-\w{4}-\w{12}`  
amzn.mws.qazwsxed-qazw-qazw-qazw-qazwsxedcrfv


